In [14]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(
    os.environ.get('WORK_DIR')) #type: ignore
sys.path.append(
    os.environ.get('DBASE_DIR')) #type: ignore
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from data import HistoricTradeDataHandler
from event import *
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import BBandsTrend2
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from backtest import OptionSignalBacktest
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
sys.path

['c:\\Users\\Zino\\python-playground\\QuantTools\\EventDriven',
 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\python310.zip',
 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\DLLs',
 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\\lib',
 'C:\\Users\\Zino\\AppData\\Local\\Microsoft\\WindowsApps\\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0',
 'c:\\Users\\Zino\\python-playground\\QuantTools\\.venv',
 '',
 'c:\\Users\\Zino\\python-playground\\QuantTools\\.venv\\lib\\site-packages',
 'c:\\Users\\Zino\\python-playground\\QuantTools\\.venv\\lib\\site-packages\\win32',
 'c:\\Users\\Zino\\python-playground\\QuantTools\\.venv\\lib\\site-packages\\win32\\lib',
 'c:\\Users\\Zino\\python-playground\\QuantTools\\.venv\\lib\\site-packages\\Pythonwin',
 'C:\\Users\\Zino\\python-playground\\QuantTools',
 'C:\\Users\\Zino\\python-play

In [16]:

def create_datasate(stocks: list, start: str,interval: str, engine: str = 'yf', timewidth = None, timeframe = None, end: str = datetime.today(), return_object = False ):
    dataset = []
    if engine.lower() == 'yf':
        for stock in stocks:
            start = prev_monday(start)
            data2 = yf.download(stock, start = start, end = end, interval=interval, progress = False)

            dataset.append(PTDataset(stock, data2))
    else:
        for stk in stocks:
            stock = Stock(stk)
            data = stock.spot(ts = True, ts_start = '2018-01-01')
            data.rename(columns = {x:x.capitalize() for x in data.columns}, inplace= True)
            data['Timestamp'] = pd.to_datetime(data['Timestamp'], format = '%Y-%m-%d')
            data2 = data.set_index('Timestamp')
            data2 = data2.asfreq('W', method = 'ffill')
            data2 = data2.fillna(0)
            data2['Next_Day_Open'] = data2.Open.shift(-1)
            data2['EMA'] = ta.ma('ema', data2.Close, length = 21).fillna(0)
            dataset.append(PTDataset(stk, data2))
    return dataset if return_object else data2


In [17]:
start, end, interval = '2023-05-29', '2024-05-28','1d'
STOCKS = ['AAPL', 'MSFT','GOOGL', 'AMD', 'AMZN']
dataset = create_datasate(STOCKS, start, interval,end = end , return_object=True)
MAStrat.start_date = pd.to_datetime('1994-03-22')
tt = PTBacktester(dataset, MAStrat, cash =1000, commission = 0.0035)
stats = tt.run()
trades = tt.trades()
shorts = tt.trades()[tt.trades()['Size'] < 0]
shorts

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration,Ticker
0,-8,37,38,121.234194,121.360001,-1.006456,-0.001038,2023-07-24,2023-07-25,None,1 days,GOOGL
0,-9,37,38,110.232833,111.139999,-8.164500,-0.008230,2023-07-24,2023-07-25,None,1 days,AMD
1,-8,39,40,111.518319,111.790001,-2.173458,-0.002436,2023-07-26,2023-07-27,None,1 days,AMD
0,-2,40,41,339.288331,333.670013,11.236635,0.016559,2023-07-27,2023-07-28,None,1 days,MSFT
2,-8,41,42,112.983167,114.160004,-9.414691,-0.010416,2023-07-28,2023-07-31,None,3 days,AMD
...,...,...,...,...,...,...,...,...,...,...,...,...
55,-4,238,239,152.594050,154.289993,-6.783774,-0.011114,2024-05-09,2024-05-10,None,1 days,AMD
56,-4,240,241,150.750519,150.429993,1.282104,0.002126,2024-05-13,2024-05-14,None,1 days,AMD
57,-4,242,243,155.045431,160.919998,-23.498267,-0.037889,2024-05-15,2024-05-16,None,1 days,AMD
59,-4,249,249,160.845069,161.410004,-2.259740,-0.003512,2024-05-24,2024-05-24,None,0 days,AMD


In [18]:
trades_ = trades[:20]
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Tag,Duration,Ticker
0,-8,37,38,121.234194,121.360001,-1.006456,-0.001038,2023-07-24,2023-07-25,None,1 days,GOOGL
0,-9,37,38,110.232833,111.139999,-8.164500,-0.008230,2023-07-24,2023-07-25,None,1 days,AMD
1,7,39,40,130.525252,131.669998,8.013221,0.008770,2023-07-26,2023-07-27,None,1 days,GOOGL
1,-8,39,40,111.518319,111.790001,-2.173458,-0.002436,2023-07-26,2023-07-27,None,1 days,AMD
0,-2,40,41,339.288331,333.670013,11.236635,0.016559,2023-07-27,2023-07-28,None,1 days,MSFT
2,-8,41,42,112.983167,114.160004,-9.414691,-0.010416,2023-07-28,2023-07-31,None,3 days,AMD
0,7,42,43,133.666197,133.550003,-0.813357,-0.000869,2023-07-31,2023-08-01,None,1 days,AMZN
1,-3,43,44,334.016837,333.630005,1.160498,0.001158,2023-08-01,2023-08-02,None,1 days,MSFT
3,-8,43,44,113.860092,119.489998,-45.039246,-0.049446,2023-08-01,2023-08-02,None,1 days,AMD
2,-3,45,46,324.859000,331.880005,-21.063015,-0.021612,2023-08-03,2023-08-04,None,1 days,MSFT


In [10]:

from EventDriven.riskmanager_async import RiskManager, close_cache, spot_cache
from pandas.tseries.offsets import BDay

rm = RiskManager(None, None, 1000000)
tick = 'GOOGL'
date = '2023-07-24'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'P'
order_settings = {'type': 'spread',
 'specifics': [{'direction': 'long',
   'rel_strike': 1.0,
   'dte': 365,
   'moneyness_width': 0.1},
  {'direction': 'short',
   'rel_strike': 0.85,
   'dte': 365,
   'moneyness_width': 0.1}],
 'name': 'vertical_spread'}

order = await rm.OrderPicker.get_order(tick, date, right, 5, order_settings)
order

                 long              short  close
10  GOOGL20240621P115  GOOGL20240621P100    5.0


{'result': 'SUCCESSFUL',
 'data': {'long': ['GOOGL20240621P115'],
  'short': ['GOOGL20240621P100'],
  'trade_id': '&L:GOOGL20240621P115&S:GOOGL20240621P100',
  'close': 5.0}}

In [19]:
#Backtest class 
evb_backtest = OptionSignalBacktest(trades_) 

In [20]:
signals = evb_backtest.bars.signal_df
signals

,Date,GOOGL,AMD,MSFT,AMZN,AAPL
0,2023-07-24,2,2,0,0,0
1,2023-07-25,-1,-1,0,0,0
2,2023-07-26,1,2,0,0,0
3,2023-07-27,-1,-1,2,0,0
4,2023-07-28,0,2,-1,0,0
5,2023-07-29,0,0,0,0,0
6,2023-07-30,0,0,0,0,0
7,2023-07-31,0,-1,0,1,0
8,2023-08-01,0,2,2,-1,0
9,2023-08-02,0,-1,-1,0,0


In [20]:
import cProfile
import pstats
import io
import asyncio

profiler = cProfile.Profile()
profiler.enable()
#run backtest
evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

        Date  GOOGL  AMD  MSFT  AMZN  AAPL
0 2023-07-24      2    2     0     0     0
Processing event: MARKET
Processing event: SIGNAL
                 long              short  close
10  GOOGL20240621P115  GOOGL20240621P100    5.0
Processing event: SIGNAL
               long            short  close
11  AMD20240621P110  AMD20240621P100   4.75
Processing event: ORDER
Processing event: ORDER
Processing event: FILL
Processing event: FILL
        Date  GOOGL  AMD  MSFT  AMZN  AAPL
1 2023-07-25     -1   -1     0     0     0
Processing event: MARKET
Processing event: SIGNAL
Processing event: SIGNAL
Processing event: ORDER
Processing event: ORDER
Processing event: FILL
Processing event: FILL
        Date  GOOGL  AMD  MSFT  AMZN  AAPL
2 2023-07-26      1    2     0     0     0
Processing event: MARKET
Processing event: SIGNAL
                 long                short  close
63  GOOGL20240621C135  GOOGL20240621C147.5   4.85
Processing event: SIGNAL
              long            short  close
3 

In [13]:
stats.print_stats(30)
print(stream.getvalue())
with open('bactest_data.txt', 'w') as f:
  stream.seek(0)
  f.write(stream.read())
  f.flush()

         22658121 function calls (22520809 primitive calls) in 195.300 seconds

   Ordered by: cumulative time
   List reduced from 2056 to 30 due to restriction <30>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  195.304   97.652 c:\Users\Zino\python-playground\QuantTools\.venv\lib\site-packages\IPython\core\interactiveshell.py:3517(run_code)
        2    0.000    0.000  195.304   97.652 {built-in method builtins.exec}
        1    0.008    0.008  195.304  195.304 c:\Users\Zino\python-playground\QuantTools\EventDriven\backtest.py:41(run)
       41    0.000    0.000  180.937    4.413 C:\Users\Zino\python-playground\QuantTools\EventDriven\portfolio.py:270(update_signal)
       41    0.008    0.000  180.935    4.413 C:\Users\Zino\python-playground\QuantTools\EventDriven\portfolio.py:224(generate_order_new)
       20    0.015    0.001  180.880    9.044 C:\Users\Zino\python-playground\QuantTools\EventDriven\riskmanager_threading.py:364

In [16]:
evb_backtest.trades

,Ticker,PnL,EntryPrice,ExitPrice,ReturnPct,Quantity,EntryTime,ExitTime,Duration,Positions
0,GOOGL,-4723.160753,499.860905,381.781886,-23.622375,40.000000,2023-07-24,2023-07-25,1,&L:GOOGL20240621P115&S:GOOGL20240621P100
1,AMD,-996.909658,447.779422,425.459160,-4.984656,44.663886,2023-08-03,2023-08-04,1,&L:AMD20240621P110&S:AMD20240621P100
2,GOOGL,1144.563971,484.982902,512.754935,5.726394,41.212827,2023-07-26,2023-07-27,1,&L:GOOGL20240621C135&S:GOOGL20240621C147.5
3,MSFT,-5346.726581,475.241332,347.671680,-26.843131,41.912214,2023-07-27,2023-07-28,1,&L:MSFT20240621P300&S:MSFT20240621P285
4,AMD,2069.049648,480.099936,529.909218,10.374774,41.539440,2023-08-01,2023-08-02,1,&L:AMD20240621P115&S:AMD20240621P105
5,AMZN,-762.519580,481.795698,463.343992,-3.829778,41.325154,2023-07-31,2023-08-01,1,&L:AMZN20240621C132.5&S:AMZN20240621C142.5
6,MSFT,2862.077146,419.193434,479.481138,14.381834,47.473646,2023-08-01,2023-08-02,1,&L:MSFT20240621P310&S:MSFT20240621P295
7,MSFT,-175.120983,290.195205,287.654310,-0.875581,68.920997,2023-08-03,2023-08-04,1,&L:MSFT20240621P300&S:MSFT20240621P290
8,AMZN,-5840.181873,492.238149,348.330771,-29.235316,40.582922,2023-08-03,2023-08-04,1,&L:AMZN20240621P117.5&S:AMZN20240621P99
9,AMD,-1410.464111,422.239117,392.402880,-7.066194,47.273526,2023-08-07,2023-08-08,1,&L:AMD20240920C115&S:AMD20240920C125


In [17]:
evb_backtest.portfolio.all_positions

## Remove commission from market value

[{'GOOGL': {},
  'AMD': {},
  'MSFT': {},
  'AMZN': {},
  'AAPL': {},
  'datetime': Timestamp('2023-07-24 00:00:00')},
 {'GOOGL': {},
  'AMD': {},
  'MSFT': {},
  'AMZN': {},
  'AAPL': {},
  'datetime': Timestamp('2023-07-24 00:00:00')},
 {'GOOGL': {},
  'AMD': {},
  'MSFT': {},
  'AMZN': {},
  'AAPL': {},
  'datetime': Timestamp('2023-07-24 00:00:00')},
 {'GOOGL': {},
  'AMD': {},
  'MSFT': {},
  'AMZN': {},
  'AAPL': {},
  'datetime': Timestamp('2023-07-24 00:00:00')},
 {'GOOGL': {},
  'AMD': {},
  'MSFT': {},
  'AMZN': {},
  'AAPL': {},
  'datetime': Timestamp('2023-07-24 00:00:00')},
 {'GOOGL': {},
  'AMD': {},
  'MSFT': {},
  'AMZN': {},
  'AAPL': {},
  'datetime': Timestamp('2023-07-24 00:00:00')},
 {'GOOGL': {'position': {'long': ['GOOGL20240621P115'],
    'short': ['GOOGL20240621P100'],
    'trade_id': '&L:GOOGL20240621P115&S:GOOGL20240621P100',
    'close': 3.8249999999999997},
   'quantity': 40.0,
   'market_value': 20000.0},
  'AMD': {},
  'MSFT': {},
  'AMZN': {},
  'AAPL':

In [85]:
evb_backtest.portfolio.options_data['GOOGL20240621P115'] - evb_backtest.portfolio.options_data['GOOGL20240621P100']

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2023-07-24,4.16,4.13,4.05,4.07,-9,-134,5.85,38,4.15,5.000,5.381509
2023-07-24,4.16,4.13,4.05,4.07,-9,-134,5.85,38,4.15,5.000,5.381509
2023-07-25,4.25,4.25,3.93,3.91,-3198,2,3.30,27,4.35,3.825,3.857051
2023-07-25,4.25,4.25,3.93,3.91,-3198,2,3.30,27,4.35,3.825,3.857051
2023-07-26,2.88,3.12,2.88,3.12,-910,18,3.02,-191,3.05,3.035,2.938961
2023-07-26,2.88,3.12,2.88,3.12,-910,18,3.02,-191,3.05,3.035,2.938961
2023-07-27,2.88,2.85,2.70,2.85,-25,-23,3.07,14,3.10,3.085,3.114296
2023-07-27,2.88,2.85,2.70,2.85,-25,-23,3.07,14,3.10,3.085,3.114296


In [24]:
from trade.helpers.helper import generate_option_tick
import pandas as pd

tick = 'AAPL'
exp = '2024-03-12'
right = 'C'
strike = 145.0
option_tick = generate_option_tick(tick, right, exp, strike)
option_tick


'AAPL20240312000145C'

In [46]:
evb_backtest.portfolio.current_positions

{'GOOGL': {}, 'AMD': {}, 'MSFT': {}, 'AMZN': {}}

In [26]:
stats.print_stats(20)  # Show the top 20 functions by cumulative time
print(stream.getvalue())

         53676519 function calls (52386882 primitive calls) in 216.194 seconds

   Ordered by: cumulative time
   List reduced from 2937 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000  216.195  108.097 /Users/chiemelienwanisobi/miniconda3/envs/openbb/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3541(run_code)
        2    0.000    0.000  216.195  108.097 {built-in method builtins.exec}
        1    0.000    0.000  216.195  216.195 /var/folders/j0/80hkbygd4lb27h9mw76gqzpw0000gn/T/ipykernel_40825/2620833521.py:1(<module>)
        1    0.003    0.003  216.195  216.195 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/backtest.py:55(run)
       21    0.000    0.000  215.775   10.275 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/portfolio.py:270(update_signal)
       21    0.003    0.000  215.775   10.275 /Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriv

In [27]:
trades = evb_backtest.portfolio.get_trades()
print(trades.to_string())

KeyError: 'option'

In [80]:
#Get all holdings
holdings = evb_backtest.get_all_holdings()
print(holdings.to_string())

               AMD    MSFT   AMZN   GOOGL   AAPL      cash  commission     total
datetime                                                                        
2024-03-05     0.0     0.0    0.0     0.0    0.0  100000.0         0.0  100000.0
2024-03-05     0.0     0.0    0.0     0.0    0.0  100000.0         0.0  100000.0
2024-03-05  3680.0     0.0    0.0     0.0    0.0   96320.0         0.0  100000.0
2024-03-05  3680.0  1624.0    0.0     0.0    0.0   94696.0         0.0  100000.0
2024-03-05  3680.0  1624.0  416.0     0.0    0.0   94280.0         0.0  100000.0
2024-03-06  4596.0  1402.0    0.0     0.0    0.0   94280.0         0.0  100278.0
2024-03-07  4804.0  1770.0    0.0     0.0    0.0   94280.0         0.0  100854.0
2024-03-08  4630.0  1880.0    0.0     0.0    0.0   94280.0         0.0  100790.0
2024-03-11  3746.0     0.0  380.0     0.0    0.0   94280.0         0.0   98406.0
2024-03-12  3310.0  1990.0  378.0     0.0    0.0   94280.0         0.0   99958.0
2024-03-13  3178.0  2096.0  

In [36]:
positions = evb_backtest.get_all_positions()
print(positions.to_string())

                             AMD                   AAPL                   MSFT                   GOOGL
datetime                                                                                              
2024-03-04                  None                   None                   None                    None
2024-03-04                  None                   None                   None                    None
2024-03-05  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-06  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-07  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-08  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-11  AMD-20240816-270.0-C                   None  MSFT-20240816-480.0-C                    None
2024-03-12  AMD-20240816-270.0-C                   None  MSFT-20240816-48

In [64]:
evb_backtest.portfolio.all_positions

[{'AMD': {'quantity': 0.0, 'option': None},
  'MSFT': {'quantity': 0.0, 'option': None},
  'AMZN': {'quantity': 0.0, 'option': None},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-04 00:00:00')},
 {'AMD': {'quantity': 0.0, 'option': None},
  'MSFT': {'quantity': 0.0, 'option': None},
  'AMZN': {'quantity': 0.0, 'option': None},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-04 00:00:00')},
 {'AMD': {'quantity': 2,
   'option': 'AMD-20240920-240.0-C',
   'entry_price': 2185.0},
  'MSFT': {'quantity': 2,
   'option': 'MSFT-20250117-540.0-C',
   'entry_price': 778.0},
  'AMZN': {'quantity': 2,
   'option': 'AMZN-20250321-205.0-C',
   'entry_price': 0.0},
  'GOOGL': {'quantity': 0.0, 'option': None},
  'AAPL': {'quantity': 0.0, 'option': None},
  'datetime': Timestamp('2024-03-05 00:00:00')},
 {'AMD': {'quantity': 2,
   'option': 'AMD-20

In [7]:
from pprint import pprint

print('All positions:')
pprint(portfolio.all_positions)

print('\nCurrent holdings:')
pprint(portfolio.current_holdings)

print('\nCurrent positions:')
pprint(portfolio.current_positions)

All positions:
[{'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': '20240226'},
 {'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': Timestamp('2024-02-26 00:00:00')},
 {'AAPL': {'option': None, 'quantity': 0.0},
  'AMD': {'option': None, 'quantity': 0.0},
  'AMZN': {'option': None, 'quantity': 0.0},
  'GOOGL': {'option': None, 'quantity': 0.0},
  'MSFT': {'option': None, 'quantity': 0.0},
  'datetime': Timestamp('2024-02-26 00:00:00')}]

Current holdings:
{'AAPL': 0.0,
 'AMD': 0.0,
 'AMZN': 0.0,
 'GOOGL': 0.0,
 'MSFT': 0.0,
 'cash': 100000,
 'commission': 0.0,
 'total': 100000}

Current positions:
{'AAPL': {'option': N

# Test RiskManager 

In [13]:
from EventDriven.riskmanager import RiskManager
from dbase.DataAPI.ThetaData import list_contracts, retrieve_option_ohlc, is_theta_data_retrieval_successful #type: ignore
import datetime
import pandas as pd
import pandas_market_calendars as mcal
import unittest
import numpy as np
import pprint as pp

tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'NFLX']


#generate date range 
nyse = mcal.get_calendar('NYSE')
year_ago_date = datetime.datetime.now() - datetime.timedelta(days=365)
schedule = nyse.schedule(start_date=year_ago_date, end_date=datetime.datetime.now())
date_range = mcal.date_range(schedule, frequency='1D')
dates = [date.strftime('%Y-%m-%d') for date in date_range]




class RiskManagerOperations(unittest.TestCase):
    def __init__(self):
        self.events = Queue(maxsize=0)  
        self.bars = HistoricTradeDataHandler(self.events, trades)
        self.risk_manager = RiskManager(bars=self.bars, events=self.events, initial_capital=1000000)
        
    def test_order_picker(self):
        ticker = np.random.choice(tickers)
        contract_date = np.random.choice(dates)
        contracts = list_contracts(ticker, pd.to_datetime(contract_date).strftime('%Y%m%d'))
        self.assertTrue(is_theta_data_retrieval_successful(contracts))
        
        contract = contracts.sample()
        print(contract)
        contract_right = contract['right'].values[0]
        contract_expiration = f"{contract['expiration'].values[0]}"
        contract_strike = float(contract['strike'].values[0])
        max_close = np.random.randint(1, 10)
        
        #order settings 
        moneyness_width = np.random.uniform(0.05, 0.1)
        rel_strike_long = np.random.uniform(1.05, 1.3) 
        rel_strike_short = np.random.uniform(0.7, 0.95)
        dte = np.random.randint(30, 365)
        
        order_settings = {
            'type': 'spread',
            'specifics': [
                {'direction': 'long', 'rel_strike': rel_strike_long, 'dte': dte, 'moneyness_width': moneyness_width},
                {'direction': 'short', 'rel_strike': rel_strike_short, 'dte': dte, 'moneyness_width': moneyness_width} 
            ],
            'name': 'vertical_spread'
        }
       
        try:
            self.order = self.risk_manager.OrderPicker.get_order(ticker, contract_expiration, contract_right, max_close, order_settings)
            self.assertIsInstance(self.order, dict)
            self.assertIsInstance(self.order['long'], list)
            self.assertIsInstance(self.order['short'], list)
            self.assertGreater(len(self.order['long']), 0)
            self.assertGreater(len(self.order['short']), 0)
            self.assertIsInstance(self.order['close'], float)
        except AssertionError as e:
            print(f"AssertionError: {e}")
            pp.pprint(contract)
            print(f"Ticker: {ticker}")
            print(f"Contract Date: {contract_date}")
            print(f"Contract Right: {contract_right}")
            print(f"Contract Expiration: {contract_expiration}")
            print(f"Contract Strike: {contract_strike}")
            print(f"Max Close: {max_close}")
            print(f"Order Settings: {order_settings}")
            raise
        except Exception as e:
            print(f"Exception: {e}")
            pp.pprint(contract)
            print(f"Ticker: {ticker}")
            print(f"Contract Date: {contract_date}")
            print(f"Contract Right: {contract_right}")
            print(f"Contract Expiration: {contract_expiration}, type: {type(contract_expiration)}")
            print(f"Contract Strike: {contract_strike}, type: {type(contract_strike)}")
            print(f"Max Close: {max_close}")
            print(f"Order Settings: {order_settings}")
            raise


ops = RiskManagerOperations()
ops.test_order_picker()

      root  expiration  strike right
3444  NVDA    20240301   270.0     C
Exception: Cannot set a DataFrame with multiple columns to the single column option_id
      root  expiration  strike right
3444  NVDA    20240301   270.0     C
Ticker: NVDA
Contract Date: 2024-02-09
Contract Right: C
Contract Expiration: 20240301, type: <class 'str'>
Contract Strike: 270.0, type: <class 'float'>
Max Close: 2
Order Settings: {'type': 'spread', 'specifics': [{'direction': 'long', 'rel_strike': 1.2305088940545148, 'dte': 284, 'moneyness_width': 0.08149699520032436}, {'direction': 'short', 'rel_strike': 0.9364378888363725, 'dte': 284, 'moneyness_width': 0.08149699520032436}], 'name': 'vertical_spread'}


ValueError: Cannot set a DataFrame with multiple columns to the single column option_id

In [15]:
from trade.assets.Stock import Stock
NVDA = Stock('NVDA', run_chain = False)

In [14]:
def generate_option_to_buy(underlier: Stock, contract_time):
        """
        Buy an option based on the underlier.
        """
        time = contract_time
        next_day_time = time + pd.DateOffset(days=1)
        print(time, next_day_time)
        option_spot = underlier.spot(ts=True, ts_start = time, ts_end = next_day_time)
        option_spot = option_spot.iloc[0] 
        stock_price = option_spot['open']#use open price as spot price on the assumption of making trades at start of day
        oom_benchmark = 0.1#10% out of the money 
        expiry_benchmark = time + pd.DateOffset(months=5)
        oom_price = stock_price * (1 + oom_benchmark)
        time_str = time.strftime("%Y%m%d")
        contracts = list_contracts(underlier.ticker, time_str)
        print(contracts)
        contracts = contracts[contracts['right'] == 'C'] 
        
        
        #Filter out contracts that are out of the money
        contracts = contracts[contracts['strike'] >= oom_price]
        
        print('comparing expiry')
        print(type(expiry_benchmark))
        print(type(contracts['expiration']))
        #filter out contracts that are  below the expiry benchmark
        contracts = contracts[pd.to_datetime(contracts['expiration'], format="%Y%m%d") >= expiry_benchmark]
        
        #select a random contract to buy
        contract = contracts.sample(n=1); 
        
        return contract
                

In [8]:
appl = Stock('AAPL')
c_time = pd.to_datetime('2024-05-06', format="%Y-%m-%d")
c_time_next = c_time + pd.DateOffset(days=1)
aapl_spot = appl.spot(ts=True, ts_start = c_time, ts_end = c_time_next)
aapl_spot.iloc[0]['open']


182.35000610351562

In [15]:
option = generate_option_to_buy(appl, c_time)
option

2024-05-06 00:00:00 2024-05-07 00:00:00
      root  expiration  strike right
0     AAPL    20241018   220.0     C
1     AAPL    20250919   215.0     C
2     AAPL    20241115   220.0     C
3     AAPL    20250321   220.0     C
4     AAPL    20250321   220.0     P
...    ...         ...     ...   ...
1060  AAPL    20240621   220.0     C
1061  AAPL    20240621   220.0     P
1062  AAPL    20240719   220.0     C
1063  AAPL    20250620   215.0     C
1064  AAPL    20240816   220.0     C

[1065 rows x 4 columns]
comparing expiry
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
<class 'pandas.core.series.Series'>


,root,expiration,strike,right
456,AAPL,20250117,235.0,C
